Model using: https://huggingface.co/Helsinki-NLP/opus-mt-en-hi

Dataset using: https://huggingface.co/datasets/cfilt/iitb-english-hindi

In [1]:
!nvidia-smi

Mon Jan 22 00:02:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# Installing hugging face and screblue for getting metrics Blue score

!pip install datasets transformers[sentencepiece] sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.4 MB/s eta 0:00:00


In [3]:
# Upgrading all the packages if there is any older packages has beed install

!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.2 MB/s eta 0:00:00
Found existing installation: transformers 4.35.2
Uninstalling transformers-4.35.2:
  Successfully uninstalled transformers-4.35.2
Found existing installation: accelerate 0.26.1
Uninstalling accelerate-0.26.1:
  Successfully uninstalled accelerate-0.26.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 19.8 MB/s eta 0:00:00
  Using cached accelerate-0.26.1-py3-none-any.whl (270 kB)


In [4]:
# importing some more libraris for fine tuning model

import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer

In [5]:
# Now we need to provide model checkpoint and dataset from hugging face. import model and dataset is done by just providing the model

model_checkpoint = "Helsinki-NLP/opus-mt-en-hi"

In [6]:
raw_datasets = load_dataset("cfilt/iitb-english-hindi")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [7]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

## In this above raw_datasets we can see that dataset has
```
training data - 1659083
validation data - 520
testing data - 2507
```

In [8]:
# let see any first data

raw_datasets['train'][0]

{'translation': {'en': 'Give your application an accessibility workout',
  'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}}

In [9]:
# Now doing word embedding through AutoTokenizer to convert text to no. to provide to model for further prediction.
# Here we are using hugging pretrain model for this word embedding

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [10]:
# Now let check id this tokenizer is working or not

tokenizer("Hello I am Sawan Yadav")

{'input_ids': [12110, 56, 489, 39336, 980, 11390, 232, 7770, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

# WHy we are getting more nos. of input_ids and attention_mask?

```Ans: Ihe input_ids often have more elements than the original words because of subword tokenization and special tokens. The attention mask has a "1" for each valid token and a "0" for each padding token, ensuring the model focuses on the relevant parts of the input.```

```Here's a step-by-step breakdown of your example:

Tokenization:
"Hello I am Sawan Yadav" is tokenized into:
[CLS], "Hello", "I", "am", "Sawa", "##n", "Yada", "##v", [SEP]

 "Sawan" might be split into subwords like "sa", "w", and "##an".

**Special Tokens**: Tokenizers often add special tokens to the sequence for specific purposes:
CLS token: Used at the beginning of the sequence to represent the overall input.
SEP token: Used to separate sentences or segments within a sequenc

Input IDs:
Each token is assigned its corresponding ID from the model's vocabulary:
[12110, 56, 489, 39336, 980, 11390, 232, 7770, 0]

Attention Mask:
All valid tokens are marked as "1", padding token as "0":
[1, 1, 1, 1, 1, 1, 1, 1, 1]
```

In [11]:
tokenizer(["Hello, this is a sentence!", "This is another sentence."])

{'input_ids': [[12110, 2, 90, 23, 19, 8800, 61, 0], [239, 23, 414, 8800, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [12]:
# Now let considet we are providing tokenizer some words we are not available in dataset, then how it is embedding that text

# "Hello": 12110
# "this": 90
# "Bappy"          --> Bappy is not in dataset

In [13]:
# Then pretrain model might seperate the text like below

##b ##a ##p ##p ##y

In [14]:
# And it might give some random no. like below:

# 'b' :2
# 'a':4
# 'p':9
# 'y':10

In [15]:
# What will the no. for text Bappy

# 24910

In [16]:
# now for hindi sentence and it text

with tokenizer.as_target_tokenizer():
    print(tokenizer(["एक्सेर्साइसर पहुंचनीयता अन्वेषक"]))

{'input_ids': [[26618, 16155, 346, 33383, 0]], 'attention_mask': [[1, 1, 1, 1, 1]]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3860: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [18]:
# Now embedding training and target data

max_input_length = 128   # it is sentence length
max_target_length = 128

source_lang = "en"
target_lang = "hi"


def preprocess_function(examples):
    inputs = [ex[source_lang] for ex in examples["translation"]]  # Seperating input and target data
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)  # tokenizing input data

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():                     # Tokenizing target data
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]   # Taking input_ids of tokenizer for target data
    return model_inputs

In [19]:
# Here it will take sometime (for me it taken almost 7 min collab)  as data size is huge inclusing all train validate and test data
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Map:   0%|          | 0/520 [00:00<?, ? examples/s]

Map:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [20]:
# Now lets load the model which we had used from hugging face

model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tf_model.h5:   0%|          | 0.00/306M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [21]:
# Parameter of this pretrained model

batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1  # We can change epochs as per the requirements and as per the system specification

In [22]:
# Now instead of providing all the data to model we will provide the data in batches and for this we need to initialize one data_colletor from DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [23]:
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

In [30]:
# Now we need to train our model for training data, validation data and testing data

train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["test"],  # Here we are using testing data for training train data as my training data is huge and it may take time for training all data
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

# train_dataset = model.prepare_tf_dataset(
#     tokenized_datasets["train"],     # We will use this for training if we much time to train all the train data
#     batch_size=batch_size,
#     shuffle=True,
#     collate_fn=data_collator,
# )

In [31]:
validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

In [32]:
generation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=8,
    shuffle=False,
    collate_fn=generation_data_collator,
)

In [33]:
# Now lets compile the model through optimizer because we are using tensorflowlibrary

optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [34]:
model.fit(train_dataset, validation_data=validation_dataset, epochs=1)

156/156 [==============================] - 84s 352ms/step - loss: 3.6511 - val_loss: 3.8684


# Interencing

In [35]:
# Now lets save the model

model.save_pretrained("tf_model/")

In [36]:
# Now lets tokenizer (for embedding new data) and model again to check how my model works for new data

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained("tf_model/")

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at tf_model/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [37]:
# Let give new data and check the tranlation prediction

input_text  = "I am learning Data science. How are you?"

tokenized = tokenizer([input_text], return_tensors='np')  # retruning numpy array
out = model.generate(**tokenized, max_length=128)
print(out)

tf.Tensor(
[[61949   104  2403  6281  2407   153   254    40   118   280    28    22
      0]], shape=(1, 13), dtype=int32)


In [38]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

मैं डाटा विज्ञान सीख रहा हूँ। आप कैसे हैं?


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3860: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [39]:
# One more time

input_text  = "My name is Sawan. How are you doing?"

tokenized = tokenizer([input_text], return_tensors='np')  # retruning numpy array
out = model.generate(**tokenized, max_length=128)
print(out)

tf.Tensor(
[[61949   500   179 21694 13017     5    40   118   280    57   160    28
     22     0]], shape=(1, 14), dtype=int32)


In [40]:
with tokenizer.as_target_tokenizer():
  print(tokenizer.decode(out[0], skip_special_tokens=True))

मेरा नाम सॉगन है। आप कैसे कर रहे हैं?


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3860: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


# By this way we can apply any translation project and we need to just change the data set checkpoint name